<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/sub/sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install focal-loss

In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [4]:
ls

cc.tr.300.vec              offenseval-tr-testset-v1.tsv   tcn.npy
cc.tr.300.vec.gz           offenseval-tr-training-v1.tsv
offenseval-annotation.txt  readme-trainingset-tr.txt


In [0]:
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
train=train.sample(frac=1)

In [7]:
train.head()

,id,tweet,subtask_a
16702,18901,@USER Arada bir tepkili ama sessizce seyretmek...,NOT
9513,30851,Telefonumun şarjı 2 gün gidiyor bataryam ölü o...,NOT
303,20378,@USER Daha dün rize düşsün diye bursanin altin...,OFF
24256,19403,@USER dediğin yere geldi hayırlısı bakalım. :),NOT
2680,22191,Unf atacaksınız neden takip ediyorsunuz.,NOT


In [0]:
train["tweet"]= train["tweet"].str.lower()
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')

In [0]:
train_labels=train['subtask_a']
train=train['tweet']
labels=le.fit_transform(train_labels)

In [10]:
train.tail()

10996    @user bunuda yaşamadik sacimi kasiyinca annem ...
11564    @user kardeşi̇m ben ne kokular çeltşm gece  e ...
7473              @user yapacak hiçbir şey yok maalesef...
23675    @user yenı nesıl cok zekı masallah cok cabuk o...
6560     hayatım boyunca tek kurmak istediğim cümle bu ...
Name: tweet, dtype: object

In [11]:
labels[0:5]

array([0, 0, 1, 0, 0])

In [12]:
import collections
collections.Counter(labels)

Counter({0: 25625, 1: 6131})

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [14]:
len(sequences_matrix),len(labels)

(31756, 31756)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [16]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.tr.300.vec.gz', 'rb') as f_in:
#     with open('cc.tr.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.tr.300.vec', word_index)


In [21]:
embeddings.shape

(10001, 300)

In [0]:
from keras.optimizers import Adam
opt=Adam(0.0001)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(50, activation="relu"))
  model.add(Dense(20, activation="relu"))
  model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [32]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.15,stratify=labels,shuffle=True)
model=model_func()
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 26992 samples, validate on 4764 samples
Epoch 1/10
 - 20s - loss: 0.1171 - acc: 0.8176 - val_loss: 0.0971 - val_acc: 0.8533
Epoch 2/10
 - 19s - loss: 0.1011 - acc: 0.8461 - val_loss: 0.0940 - val_acc: 0.8604
Epoch 3/10
 - 19s - loss: 0.0978 - acc: 0.8526 - val_loss: 0.0927 - val_acc: 0.8608
Epoch 4/10
 - 19s - loss: 0.0956 - acc: 0.8573 - val_loss: 0.0913 - val_acc: 0.8640
Epoch 5/10
 - 19s - loss: 0.0927 - acc: 0.8616 - val_loss: 0.0923 - val_acc: 0.8594
Epoch 6/10
 - 19s - loss: 0.0909 - acc: 0.8659 - val_loss: 0.0928 - val_acc: 0.8583
Epoch 7/10
 - 19s - loss: 0.0891 - acc: 0.8691 - val_loss: 0.0902 - val_acc: 0.8629
Epoch 8/10
 - 19s - loss: 0.0865 - acc: 0.8729 - val_loss: 0.0899 - val_acc: 0.8654
Epoch 9/10
 - 19s - loss: 0.0845 - acc: 0.8761 - val_loss: 0.0911 - val_acc: 0.8652
Epoch 10/10
 - 19s - loss: 0.0819 - acc: 0.8802 - val_loss: 0.0906 - val_acc: 0.8642


In [33]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

4764/4764 [==============================] - 1s 227us/step
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3844
           1       0.74      0.45      0.56       920

    accuracy                           0.86      4764
   macro avg       0.81      0.71      0.74      4764
weighted avg       0.85      0.86      0.85      4764



In [34]:
test=pd.read_csv( 'offenseval-tr-testset-v1.tsv',sep='\t',quoting=csv.QUOTE_NONE,encoding='utf-8')
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()

test.head()

,id,tweet
0,41993,sayın başkanım bu şekilde devam inşallah👏
1,23000,"herkes gevşekliği kadar duyar kasıyor,hayat bö..."
2,42478,olgun ilişkisi olan arkadaş size en güzel hedi...
3,21748,burada atıp tutacağına o kötü koşullarda ku...
4,13607,i̇şte o onur dediğin sende yok sorun o işte


In [0]:
X_test=test['tweet']

test_sequences = tokenize.texts_to_sequences(X_test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [36]:
y_test = model.predict(X_test_sequences, batch_size=64, verbose=1)


3528/3528 [==============================] - 1s 195us/step


In [0]:
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

In [41]:
y_test=le.inverse_transform(y_test)
y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([test['id'], y_test['label']], axis=1)
y_test.head()


,id,label
0,41993,NOT
1,23000,NOT
2,42478,NOT
3,21748,NOT
4,13607,NOT


In [0]:
y_test.to_csv('/content/sub1.csv',index=False,header=None)